# Fortune / Great Place to Work “100 Best” Lists → CSV (Beautiful Soup)

###### Scalling up!

In [106]:
import re
import time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

# -------------------------
# 0) Helpers
# -------------------------

DEFAULT_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
}

def gvkey6(x):
    if pd.isna(x):
        return pd.NA
    s = str(x).strip()
    s = "".join(ch for ch in s if ch.isdigit())
    return s.zfill(6) if s else pd.NA

def make_match_key(name: str) -> str:
    if not isinstance(name, str):
        return ""
    n = name.lower()
    n = re.sub(r"\(.*?\)", "", n)      # remove parentheses
    n = n.replace("&", "and")
    n = re.sub(r"[^a-z0-9\s]", " ", n) # remove punctuation
    suffixes = {
        "inc", "incorporated", "corp", "corporation", "co", "company",
        "llc", "llp", "lp", "plc", "ltd", "limited",
        "group", "holdings"
    }
    tokens = [t for t in n.split() if t and t not in suffixes]
    return " ".join(tokens).strip()

def normalize_whitespace(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def parse_rank_company_pairs_from_text(soup: BeautifulSoup):
    raw_lines = [normalize_whitespace(x) for x in soup.get_text("\n").splitlines()]
    lines = [x for x in raw_lines if x]

    results = []
    i = 0
    while i < len(lines) - 1:
        if re.fullmatch(r"\d{1,3}", lines[i]):
            rank = int(lines[i])
            name = lines[i + 1].strip()
            if 1 <= rank <= 100 and not re.fullmatch(r"\d{1,3}", name):
                results.append((rank, name))
                i += 2
                continue
        i += 1

    # keep first company seen per rank
    dedup = {}
    for r, n in results:
        if r not in dedup:
            dedup[r] = n
    return sorted(dedup.items(), key=lambda x: x[0])

def scrape_gptw_from_url(url: str, year: int | None = None, sleep_s: float = 0.75) -> pd.DataFrame:
    if year is None:
        m = re.search(r"/(\d{4})(?:/)?$", url)
        if not m:
            raise ValueError("Could not infer year from URL; pass year=YYYY explicitly.")
        year = int(m.group(1))

    time.sleep(sleep_s)
    html = requests.get(url, headers=DEFAULT_HEADERS, timeout=30).text
    soup = BeautifulSoup(html, "lxml")
    pairs = parse_rank_company_pairs_from_text(soup)

    df_raw = pd.DataFrame(pairs, columns=["rank", "company_name"])
    df_raw.insert(0, "year", year)
    return df_raw

# -------------------------
# 1) Build a Compustat name index ONCE
# -------------------------

def build_compustat_name_pool(db):
    """
    Pull Compustat company names and build a long 'name_pool':
    (gvkey, name_key, source_name, sic)

    Adds filters to avoid funds/ETFs/trusts/etc.
    """
    comp_cols = db.describe_table(library="comp", table="company")
    comp_cols = [c.lower() for c in comp_cols["name"].tolist()]

    base_cols = ["gvkey", "conm", "conml", "sic"]
    select_cols = [c for c in base_cols if c in comp_cols]

    sql = f"""
    SELECT {", ".join(select_cols)}
    FROM comp.company
    WHERE gvkey IS NOT NULL
    """
    comp = db.raw_sql(sql)
    comp.columns = [c.lower() for c in comp.columns]
    comp["gvkey"] = comp["gvkey"].apply(gvkey6)

    # ---- normalize SIC to numeric-ish string for filtering
    if "sic" in comp.columns:
        comp["sic"] = pd.to_numeric(comp["sic"], errors="coerce")
    else:
        comp["sic"] = pd.NA

    # =========================================================
    # 1) HARD FILTER: drop investment vehicles (SIC 67xx)
    # =========================================================
    # SIC 67xx includes investment offices, funds, trusts, ETFs, etc.
    comp = comp[~comp["sic"].between(6700, 6799, inclusive="both") | comp["sic"].isna()].copy()

    # =========================================================
    # 2) SOFT FILTER: drop names that look like funds/ETFs/trusts
    # =========================================================
    bad_name_re = re.compile(
        r"\b(etf|funds?|trust|income|portfolio|index|notes?|"
        r"depositary|adr|etn|closed[-\s]?end|open[-\s]?end|"
        r"spac|acquisition\s+corp|holdings?\s+trust)\b",
        flags=re.IGNORECASE,
    )

    for col in ["conm", "conml"]:
        if col in comp.columns:
            comp[col] = comp[col].fillna("")
            comp = comp[~comp[col].str.contains(bad_name_re, na=False)].copy()
        else:
            comp[col] = ""

    # ---- build match keys
    comp["conm_key"]  = comp["conm"].apply(make_match_key)  if "conm"  in comp.columns else ""
    comp["conml_key"] = comp["conml"].apply(make_match_key) if "conml" in comp.columns else ""

    name_pool = pd.concat(
        [
            comp[["gvkey", "conm", "conm_key", "sic"]].rename(
                columns={"conm": "source_name", "conm_key": "name_key"}
            ),
            comp[["gvkey", "conml", "conml_key", "sic"]].rename(
                columns={"conml": "source_name", "conml_key": "name_key"}
            ),
        ],
        axis=0,
        ignore_index=True,
    )

    name_pool = name_pool.dropna(subset=["gvkey"])
    name_pool["name_key"] = name_pool["name_key"].astype(str)
    name_pool = name_pool[name_pool["name_key"].str.len() > 0]
    name_pool = name_pool.drop_duplicates(subset=["gvkey", "name_key"])

    return name_pool

# -------------------------
# 2) High-confidence matcher ONLY
# -------------------------

def match_gvkey_high_confidence(
    df_names: pd.DataFrame,
    name_pool: pd.DataFrame,
    ok_cutoff: float = 92.0,
    review_cutoff: float = 85.0,
):
    """
    Assign gvkey only when score >= ok_cutoff.
    Keep REVIEW/NO_MATCH for transparency; but gvkey stays NA unless OK.
    """
    try:
        from rapidfuzz import process, fuzz
    except Exception:
        raise ImportError(
            "rapidfuzz is required. In Jupyter run: pip install rapidfuzz"
        )

    choices = name_pool["name_key"].tolist()

    out_gvkey = []
    out_matched_key = []
    out_score = []
    out_flag = []

    for mk in df_names["match_key"].fillna("").astype(str):
        if not mk:
            out_gvkey.append(pd.NA)
            out_matched_key.append(pd.NA)
            out_score.append(0.0)
            out_flag.append("NO_MATCH")
            continue

        hit = process.extractOne(mk, choices, scorer=fuzz.token_set_ratio)
        if hit is None:
            out_gvkey.append(pd.NA)
            out_matched_key.append(pd.NA)
            out_score.append(0.0)
            out_flag.append("NO_MATCH")
            continue

        matched_key, score, idx = hit
        score = float(score)

        if score >= ok_cutoff:
            gv = name_pool.iloc[idx]["gvkey"]
            out_gvkey.append(gv)
            out_flag.append("OK")
        elif score >= review_cutoff:
            out_gvkey.append(pd.NA)  # IMPORTANT: do not assign
            out_flag.append("REVIEW")
        else:
            out_gvkey.append(pd.NA)  # IMPORTANT: do not assign
            out_flag.append("NO_MATCH")

        out_matched_key.append(matched_key)
        out_score.append(score)

    df = df_names.copy()
    df["gvkey"] = out_gvkey
    df["comp_name_key_matched"] = out_matched_key
    df["name_match_score"] = out_score
    df["match_flag"] = out_flag

    # bring SIC for OK matches (optional, but useful)
    df = df.merge(
        name_pool[["gvkey", "sic"]].drop_duplicates("gvkey"),
        on="gvkey",
        how="left",
    )

    return df

# -------------------------
# 3) One-call function for a year (scrape -> high-confidence gvkey)
# -------------------------

def scrape_and_match_gvkey(url: str, db, name_pool: pd.DataFrame, ok_cutoff=92.0, review_cutoff=85.0):
    df = scrape_gptw_from_url(url)
    df["name_raw"] = df["company_name"]
    df["match_key"] = df["company_name"].apply(make_match_key)

    df = match_gvkey_high_confidence(
        df,
        name_pool=name_pool,
        ok_cutoff=ok_cutoff,
        review_cutoff=review_cutoff,
    )
    return df

In [107]:
import wrds
from rapidfuzz import process, fuzz

db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [108]:
# -------------------------
# USAGE
# -------------------------
# Run ONCE:
name_pool = build_compustat_name_pool(db)

Approximately 56070 rows in comp.company.


/var/folders/bb/rsmxjk4n2k7_166l026d8h400000gn/T/ipykernel_28414/390345803.py:136: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  comp = comp[~comp[col].str.contains(bad_name_re, na=False)].copy()
/var/folders/bb/rsmxjk4n2k7_166l026d8h400000gn/T/ipykernel_28414/390345803.py:136: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  comp = comp[~comp[col].str.contains(bad_name_re, na=False)].copy()


In [109]:
df_2025 = scrape_and_match_gvkey("https://www.greatplacetowork.com/best-workplaces/100-best/2025", db, name_pool)

In [134]:
# Now we clean

def clean_df(
    df,
    private_companies,
    manual_gvkey_overrides,
    validate=True,
):
    df = df.copy()

    # Ensure 'public' exists
    if "public" not in df.columns:
        df["public"] = np.nan

    # -------------------------
    # 1) Force private companies
    # -------------------------
    private_set = {str(x).upper().strip() for x in private_companies}

    mask_private = (
        df["company_name"]
        .astype(str)
        .str.upper()
        .str.strip()
        .isin(private_set)
    )

    df.loc[mask_private, "gvkey"] = np.nan
    df.loc[mask_private, "public"] = 0
    df.loc[mask_private, "match_flag"] = "MANUAL_PRIVATE"

    # -------------------------
    # 2) Manual GVKEY overrides
    # -------------------------
    for name, vals in manual_gvkey_overrides.items():
        nm = str(name).upper().strip()

        mask = (
            df["company_name"]
            .astype(str)
            .str.upper()
            .str.strip()
            .eq(nm)
        )

        df.loc[mask, "gvkey"] = vals["gvkey"]
        if "conm" in df.columns and "conm" in vals:
            df.loc[mask, "conm"] = vals["conm"]

        df.loc[mask, "match_flag"] = "MANUAL_OVERRIDE"
        df.loc[mask, "public"] = 1  # overrides are meant to be public

    # -------------------------
    # 3) Final public flag (ONLY fill missing)
    # -------------------------
    mask_public_missing = df["public"].isna()
    df.loc[mask_public_missing, "public"] = np.where(
        df.loc[mask_public_missing, "gvkey"].notna(), 1, 0
    )

    # -------------------------
    # 4) Optional validation
    # -------------------------
    if validate:
        bad_private = df.loc[
            (df["public"] == 0) & df["gvkey"].notna(),
            ["company_name", "gvkey", "match_flag"],
        ]
        bad_public = df.loc[
            (df["public"] == 1) & df["gvkey"].isna(),
            ["company_name", "match_flag"],
        ]

        if not bad_private.empty or not bad_public.empty:
            raise ValueError(
                "Public/private consistency check failed.\n\n"
                f"Private with GVKEY:\n{bad_private}\n\n"
                f"Public without GVKEY:\n{bad_public}"
            )

    return df

In [135]:
manual_gvkey_overrides = {
    "TARGET CORPORATION": {
        "gvkey": "002572",   # Target Corp (retail)
        "conm": "TARGET CORP"
    },
    "BOX, INC.": {
        "gvkey": "012141",   # Box Inc (cloud software)
        "conm": "BOX INC"
    },
    "DOW": {
        "gvkey": "018257",   # Dow Inc (ticker: DOW)
        "conm": "DOW INC"
    },
}

In [136]:
private_companies = [
    "Wegmans Food Markets, Inc.",
    "World Wide Technology",
    "David Weekley Homes",
    "PricewaterhouseCoopers LLP",
    "Plante Moran",
    "Kimley-Horn",
    "Deloitte",
    "Nationwide Mutual Insurance Company",
    "Bain & Company, Inc.",
    "Sheetz, Inc.",
    "OneDigital",
    "Trek Bicycle",
    "Panda Restaurant Group Inc.",
    "EY",
    "RSM US LLP",
    "Baptist Health South Florida",
    "BDO USA",
    "KPMG LLP",
    "Protiviti",
    "The Wonderful Company",
    "Allianz Life",
    "SCAN Health Plan",
    "Perkins Coie LLP",
    "PCL Construction",
    "TP (including Alliance One, Health Advocate, Language Line Solutions, PSG Global Solutions, Senture, and TLScontact)",
    "Wellstar Health System",
    "Hilti, Inc",
    "Jackson Healthcare",
    "Publix Super Markets",
    "CHG Healthcare Services",
    "Veterans United Home Loans",
    "Atlantic Health System",
    "Modern Technology Solutions, Inc.",
    "Ryan, LLC",
    "Alston & Bird LLP",
    "Baird",

    # 🔴 added now (previously leaking gvkeys)
    "Power Home Remodeling",
    "DHL Express",
    "IGS Energy",
    "New American Funding",
]

In [ ]:
df_2025_clean = clean_df(
    df_2025,
    private_companies=private_companies,
    manual_gvkey_overrides=manual_gvkey_overrides
)
# Now let's sum the 1 in the public column: 

print(df_2025_clean["public"].sum())

df_2025_clean.to_excel("great_place_to_work_2025_final.xlsx", index=False)

### Now we start building up for the previous years:

In [145]:
WARM_COLS = ["match_key", "gvkey", "sic", "public", "company_name"]

def init_master_warm(df_clean):
    """
    Start the master warm table from a cleaned year df.
    Keeps only rows with a gvkey, one per match_key.
    """
    cols = [c for c in WARM_COLS if c in df_clean.columns]
    master = (
        df_clean
        .dropna(subset=["match_key", "gvkey"])
        .sort_values(["match_key", "year", "rank"])
        .drop_duplicates("match_key", keep="last")
        [cols]
        .copy()
    )
    return master

def update_master_warm(master_warm, df_new_clean):
    """
    Add/overwrite match_key rows using the newest cleaned df.
    New df wins when match_key conflicts.
    """
    cols = [c for c in WARM_COLS if c in df_new_clean.columns]
    add = (
        df_new_clean
        .dropna(subset=["match_key", "gvkey"])
        .sort_values(["match_key", "year", "rank"])
        .drop_duplicates("match_key", keep="last")
        [cols]
        .copy()
    )

    master = pd.concat([master_warm[cols], add], ignore_index=True)
    master = master.drop_duplicates("match_key", keep="last")  # newest wins
    return master

In [146]:
def apply_warm_start_from_master(df_year_raw, master_warm):
    """
    Fill gvkey/sic/public/conm for any match_key found in master_warm.
    """
    df = df_year_raw.copy()

    # ensure output cols exist
    for c in ["gvkey", "sic", "public", "match_flag", "company_name"]:
        if c not in df.columns:
            df[c] = np.nan

    warm_map = master_warm.set_index("match_key").to_dict(orient="index")

    for i, mk in df["match_key"].fillna("").items():
        if mk in warm_map:
            for k, v in warm_map[mk].items():
                df.at[i, k] = v
            df.at[i, "match_flag"] = "WARM_START_MASTER"

    return df

In [147]:
def run_year_cumulative(
    url,
    db,
    name_pool,
    master_warm,
    private_companies,
    manual_gvkey_overrides,
    ok_cutoff=92.0,
    review_cutoff=85.0,
    validate=True,
):
    # 1) scrape
    df = scrape_gptw_from_url(url)
    df["name_raw"] = df["company_name"]
    df["match_key"] = df["company_name"].apply(make_match_key)

    # 2) warm start from ALL prior cleaned years
    if master_warm is not None and len(master_warm) > 0:
        df = apply_warm_start_from_master(df, master_warm)

    # 3) force private + manual overrides BEFORE fuzzy
    df = clean_df(
        df,
        private_companies=private_companies,
        manual_gvkey_overrides=manual_gvkey_overrides,
        validate=False,
    )

    # 4) fuzzy match only for rows still missing gvkey AND not forced private
    needs = (df["gvkey"].isna()) & (df["match_flag"] != "MANUAL_PRIVATE")
    if needs.any():
        df_matched = match_gvkey_high_confidence(
            df.loc[needs, ["year","rank","company_name","name_raw","match_key"]].copy(),
            name_pool=name_pool,
            ok_cutoff=ok_cutoff,
            review_cutoff=review_cutoff,
        )
        for col in ["gvkey", "sic", "comp_name_key_matched", "name_match_score", "match_flag"]:
            if col in df_matched.columns:
                df.loc[needs, col] = df_matched[col].values

    # 5) final public flag (don’t override MANUAL_PRIVATE)
    mask_pub_missing = df["public"].isna()
    df.loc[mask_pub_missing, "public"] = np.where(df.loc[mask_pub_missing, "gvkey"].notna(), 1, 0)

    # 6) validate (optional)
    if validate:
        bad_private = df[(df["match_flag"] == "MANUAL_PRIVATE") & df["gvkey"].notna()]
        if not bad_private.empty:
            raise ValueError("Some MANUAL_PRIVATE rows still have gvkey assigned.")

    return df

In [148]:
# start with 2025 (already cleaned)
master_warm = init_master_warm(df_2025_clean)

# run 2024
df_2024_clean = run_year_cumulative(
    "https://www.greatplacetowork.com/best-workplaces/100-best/2024",
    db=db,
    name_pool=name_pool,
    master_warm=master_warm,
    private_companies=private_companies,
    manual_gvkey_overrides=manual_gvkey_overrides,
)

# update master with 2024 results
master_warm = update_master_warm(master_warm, df_2024_clean)

# run 2023 (now warm-starts from BOTH 2025 + 2024)
df_2023_clean = run_year_cumulative(
    "https://www.greatplacetowork.com/best-workplaces/100-best/2023",
    db=db,
    name_pool=name_pool,
    master_warm=master_warm,
    private_companies=private_companies,
    manual_gvkey_overrides=manual_gvkey_overrides,
)

master_warm = update_master_warm(master_warm, df_2023_clean)

/var/folders/bb/rsmxjk4n2k7_166l026d8h400000gn/T/ipykernel_28414/597450935.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '005643' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, k] = v
/var/folders/bb/rsmxjk4n2k7_166l026d8h400000gn/T/ipykernel_28414/597450935.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'WARM_START_MASTER' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, "match_flag"] = "WARM_START_MASTER"
/var/folders/bb/rsmxjk4n2k7_166l026d8h400000gn/T/ipykernel_28414/2998323198.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<IntegerArray>
[<NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, <NA>, 7373, <NA>, 6153,
 <

In [149]:
# now we save them to excel 

df_2024_clean.to_excel("great_place_to_work_2024_final.xlsx", index=False)
df_2023_clean.to_excel("great_place_to_work_2023_final.xlsx", index=False)

In [150]:
df_2024_clean["gvkey"].notna().sum()

53

In [151]:
df_2023_clean["gvkey"].notna().sum()

58